# Load and cache data

In [1]:
val df = spark.read.format("csv")
  .option("inferSchema", "true")
  .option("header", "true")
  .load("LCLoan.csv")

Intitializing Scala interpreter ...

Spark Web UI available at http://c080bc5f4c74:4040
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1558778484533)
SparkSession available as 'spark'


df: org.apache.spark.sql.DataFrame = [id: string, member_id: string ... 143 more fields]


In [2]:
df.cache
df.count

2019-05-25 10:02:11,785 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


res0: Long = 2260668


# Register temp table for use in Spark SQL

In [7]:
df.createOrReplaceTempView("df")

In [ ]:
spark.sql("select * from df").show(5,false)

# Drop fields with population rate lower than 90%

In [42]:
// get all columns
val selectColumns = df.columns.toArray

selectColumns: Array[String] = Array(id, member_id, loan_amnt, funded_amnt, funded_amnt_inv, term, int_rate, installment, grade, sub_grade, emp_title, emp_length, home_ownership, annual_inc, verification_status, issue_d, loan_status, pymnt_plan, url, desc, purpose, title, zip_code, addr_state, dti, delinq_2yrs, earliest_cr_line, inq_last_6mths, mths_since_last_delinq, mths_since_last_record, open_acc, pub_rec, revol_bal, revol_util, total_acc, initial_list_status, out_prncp, out_prncp_inv, total_pymnt, total_pymnt_inv, total_rec_prncp, total_rec_int, total_rec_late_fee, recoveries, collection_recovery_fee, last_pymnt_d, last_pymnt_amnt, next_pymnt_d, last_credit_pull_d, collections_12_mths_ex_med, mths_since_last_major_derog, policy_code, application_type, annual_inc_joint, dti_joint, v...

In [43]:
// Whole file count
val fileCount = df.count

fileCount: Long = 2260668


In [38]:
// Threshold
val sparseThreshold = 0.9

sparseThreshold: Double = 0.9


In [44]:
// Get the list of good columns
var goodColumns = Array[String]()
for (s <- selectColumns) {
    
    // get the counts of not null values for give column
    val sqlText = s"select count(1) as cnt from df where ${s} is not null"
    val not_null_count = spark.sql(sqlText)
    val not_null_int = not_null_count.select("cnt").collect().map(_(0)).toList(0).toString.toDouble
    
    // output the columnss with population rate above threshold
    val populationRate = not_null_int / fileCount
    if ( populationRate >= sparseThreshold) {
        println(s"${s}: ${not_null_int}: ${populationRate}")
        goodColumns = goodColumns :+ s
    }
}

loan_amnt: 2260668.0: 1.0
funded_amnt: 2260668.0: 1.0
funded_amnt_inv: 2260668.0: 1.0
term: 2260668.0: 1.0
int_rate: 2260668.0: 1.0
installment: 2260668.0: 1.0
grade: 2260668.0: 1.0
sub_grade: 2260668.0: 1.0
emp_title: 2093737.0: 0.9261585513662334
emp_length: 2260668.0: 1.0
home_ownership: 2260668.0: 1.0
annual_inc: 2260664.0: 0.999998230611483
verification_status: 2260668.0: 1.0
issue_d: 2260668.0: 1.0
loan_status: 2260668.0: 1.0
pymnt_plan: 2260668.0: 1.0
purpose: 2260667.0: 0.9999995576528707
title: 2237345.0: 0.9896831379043717
zip_code: 2260666.0: 0.9999991153057415
addr_state: 2260667.0: 0.9999995576528707
dti: 2258956.0: 0.9992427017147144
delinq_2yrs: 2260638.0: 0.9999867295861223
earliest_cr_line: 2260638.0: 0.9999867295861223
inq_last_6mths: 2260637.0: 0.9999862872389931
open_acc: 2260558.0: 0.9999513418157819
pub_rec: 2260576.0: 0.9999593040641085
revol_bal: 2260614.0: 0.9999761132550202
revol_util: 2258834.0: 0.9991887353649452
total_acc: 2260618.0: 0.9999778826435373
init

goodColumns: Array[String] = Array(loan_amnt, funded_amnt, funded_amnt_inv, term, int_rate, installment, grade, sub_grade, emp_title, emp_length, home_ownership, annual_inc, verification_status, issue_d, loan_status, pymnt_plan, purpose, title, zip_code, addr_state, dti, delinq_2yrs, earliest_cr_line, inq_last_6mths, open_acc, pub_rec, revol_bal, revol_util, total_acc, initial_list_status, out_prncp, out_prncp_inv, total_pymnt, total_pymnt_inv, total_rec_prncp, total_rec_int, total_rec_late_fee, recoveries, collection_recovery_fee, last_pymnt_d, last_pymnt_amnt, last_credit_pull_d, collections_12_mths_ex_med, policy_code, application_type, acc_now_delinq, tot_coll_amt, tot_cur_bal, total_rev_hi_lim, acc_open_past_24mths, avg_cur_bal, bc_open_to_buy, bc_util, chargeoff_within_12_mths, de...

In [45]:
// Column number reduced to 86
goodColumns.size

res22: Int = 86


In [46]:
val goodColumnsStr = goodColumns.mkString(",")

goodColumnsStr: String = loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_re...

In [47]:
val dfGoodColumn = spark.sql(s"select ${goodColumnsStr} from df")

dfGoodColumn: org.apache.spark.sql.DataFrame = [loan_amnt: int, funded_amnt: int ... 84 more fields]


In [31]:
dfGoodColumn.printSchema

root
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullab

In [49]:
dfGoodColumn.createOrReplaceTempView("dfGoodColumn")

# Retain only 2017 data

In [50]:
val df_2017 = spark.sql("select * from dfGoodColumn where issue_d like '%2017'")

df_2017: org.apache.spark.sql.DataFrame = [loan_amnt: int, funded_amnt: int ... 84 more fields]


In [56]:
// Reduced count to 443579
df_2017.count

res29: Long = 443579


In [55]:
// Verify all records from 2017
df_2017.select("issue_d").distinct.show(false)

+--------+
|issue_d |
+--------+
|Sep-2017|
|Jun-2017|
|Nov-2017|
|Feb-2017|
|Dec-2017|
|Aug-2017|
|May-2017|
|Jul-2017|
|Jan-2017|
|Oct-2017|
|Mar-2017|
|Apr-2017|
+--------+



In [57]:
df_2017.repartition(1).write.csv("LCLoan_2017.csv")

#### Please ignore below

#### Some random checks

In [97]:
spark.sql("select debt_settlement_flag,count(1) from dfGoodColumn group by debt_settlement_flag order by 2 desc").show(false)

+--------------------+--------+
|debt_settlement_flag|count(1)|
+--------------------+--------+
|N                   |2227363 |
|Y                   |33055   |
|null                |193     |
|Cash                |57      |
+--------------------+--------+



In [98]:
spark.sql("select loan_status,count(1) from dfGoodColumn group by loan_status order by 2 desc").show(false)

+---------------------------------------------------+--------+
|loan_status                                        |count(1)|
+---------------------------------------------------+--------+
|Fully Paid                                         |1041952 |
|Current                                            |919695  |
|Charged Off                                        |261654  |
|Late (31-120 days)                                 |21897   |
|In Grace Period                                    |8952    |
|Late (16-30 days)                                  |3737    |
|Does not meet the credit policy. Status:Fully Paid |1988    |
|Does not meet the credit policy. Status:Charged Off|761     |
|Default                                            |31      |
|Oct-2015                                           |1       |
+---------------------------------------------------+--------+



#### Testing Robert's sampling method

In [99]:
val fractions = Map("Default" -> 1.0)

fractions: scala.collection.immutable.Map[String,Double] = Map(Default -> 1.0)


In [101]:
val subsetDF_default = dfGoodColumn.stat.sampleBy("loan_status", fractions, 36L)

subsetDF_default: org.apache.spark.sql.DataFrame = [loan_amnt: int, funded_amnt: int ... 84 more fields]


In [102]:
subsetDF_default.createOrReplaceTempView("subsetDF_default")

In [103]:
spark.sql("select loan_status,count(1) from subsetDF_default group by loan_status order by 2 desc").show(false)

+-----------+--------+
|loan_status|count(1)|
+-----------+--------+
|Default    |31      |
+-----------+--------+



It seems we need to specify all values in column load_status, other wise they will be treated as 0

more here

https://spark.apache.org/docs/1.6.0/api/java/org/apache/spark/sql/DataFrameStatFunctions.html#sampleBy(java.lang.String,%20java.util.Map,%20long)

#### Checking where is all defaults coming from

In [104]:
spark.sql("select issue_d,count(1) from dfGoodColumn where loan_status = 'Default' group by issue_d order by 2 desc").show(false)

+--------+--------+
|issue_d |count(1)|
+--------+--------+
|Mar-2016|4       |
|Apr-2018|3       |
|Apr-2017|2       |
|May-2017|2       |
|Mar-2018|2       |
|Jan-2016|2       |
|Apr-2016|2       |
|Jul-2018|2       |
|Oct-2016|1       |
|Sep-2017|1       |
|Jun-2017|1       |
|Sep-2016|1       |
|Dec-2017|1       |
|Dec-2016|1       |
|Jan-2017|1       |
|Jun-2018|1       |
|Feb-2016|1       |
|Oct-2017|1       |
|May-2018|1       |
|Mar-2017|1       |
+--------+--------+

